In [77]:
import pandas as pd

etch = [550, 604, 669, 650, 633, 601, 642, 635, 1037, 1052, 749, 868, 1075, 1063, 729, 860]
replicate = [1, 2]*8
A = [-1, -1, 1, 1]*4
B = [-1, -1, -1, -1, 1, 1, 1, 1]*2
C = [-1]*8 + [1]*8

data = pd.DataFrame({'Rate': etch, 'A': A, 'B': B, 'C': C, 'Replicate': replicate})
data['SqRate'] = data.Rate**2
data

print((data.Rate.sum()**2)/8, data.SqRate.sum())

19272736.125 10167789


In [70]:
rep1 = []
rep2 = []
i = 0
for entry in etch:
    if (i%2 == 0):
        rep1.append(entry)
        A[i] = 0
        B[i] = 0
        C[i] = 0
    else:
        rep2.append(entry)
    i += 1

A = [x for x in A if x!=0]
B = [x for x in B if x!=0]
C = [x for x in C if x!=0]

new_data = pd.DataFrame({'A': A, 'B': B, 'C': C, 'Rep1': rep1, 'Rep2': rep2})
new_data['Total'] = new_data.Rep1 + new_data.Rep2

In [71]:
new_data

,A,B,C,Rep1,Rep2,Total
0,-1,-1,-1,550,604,1154
1,1,-1,-1,669,650,1319
2,-1,1,-1,633,601,1234
3,1,1,-1,642,635,1277
4,-1,-1,1,1037,1052,2089
5,1,-1,1,749,868,1617
6,-1,1,1,1075,1063,2138
7,1,1,1,729,860,1589


## Manual Calculation

In [106]:
n = 2
effect = {}

effect['A'] = (new_data.query('A == 1').Total.sum() - new_data.query('A == -1').Total.sum())/(4*n)
effect['B'] = (new_data.query('B == 1').Total.sum() - new_data.query('B == -1').Total.sum())/(4*n)
effect['C'] = (new_data.query('C == 1').Total.sum() - new_data.query('C == -1').Total.sum())/(4*n)

effect['AB'] = (new_data.query('(A+B) != 0').Total.sum() - new_data.query('(A+B) == 0').Total.sum())/(4*n)
effect['AC'] = (new_data.query('(A+C) != 0').Total.sum() - new_data.query('(A+C) == 0').Total.sum())/(4*n)
effect['BC'] = (new_data.query('(B+C) != 0').Total.sum() - new_data.query('(B+C) == 0').Total.sum())/(4*n)

effect['ABC'] = (new_data.query('((A+B+C) == -1)|((A+B+C) == 3)').Total.sum() - new_data.query('((A+B+C) == 1)|((A+B+C) == -3)').Total.sum())/(4*n)

effect

{'A': -101.625,
 'AB': -24.875,
 'ABC': 5.625,
 'AC': -153.625,
 'B': 7.375,
 'BC': -2.125,
 'C': 306.125}

In [108]:
n = 2
effect_1 = {}

effect_1['A'] = (new_data.query('A == 1').Total.sum() - new_data.query('A == -1').Total.sum())/(4*n)
effect_1['B'] = (new_data.query('B == 1').Total.sum() - new_data.query('B == -1').Total.sum())/(4*n)
effect_1['C'] = (new_data.query('C == 1').Total.sum() - new_data.query('C == -1').Total.sum())/(4*n)

effect_1['AB'] = (new_data.query('(A*B) == 1').Total.sum() - new_data.query('(A*B) == -1').Total.sum())/(4*n)
effect_1['AC'] = (new_data.query('(A*C) == 1').Total.sum() - new_data.query('(A*C) == -1').Total.sum())/(4*n)
effect_1['BC'] = (new_data.query('(B*C) == 1').Total.sum() - new_data.query('(B*C) == -1').Total.sum())/(4*n)

effect_1['ABC'] = (new_data.query('(A*B*C) == 1').Total.sum() - new_data.query('(A*B*C) == -1').Total.sum())/(4*n)

effect_1

{'A': -101.625,
 'AB': -24.875,
 'ABC': 5.625,
 'AC': -153.625,
 'B': 7.375,
 'BC': -2.125,
 'C': 306.125}

In [85]:
SS = {x:((effect[x]*(4*n))**2)/(8*n) for x in effect.keys()}

new_data['SqTotal'] = new_data.Rep1**2 + new_data.Rep2**2
SS['Total'] = new_data.SqTotal.sum() - (new_data.Total.sum()**2)/(8*n)
SS

{'A': 41310.5625,
 'AB': 2475.0625,
 'ABC': 126.5625,
 'AC': 94402.5625,
 'B': 217.5625,
 'BC': 18.0625,
 'C': 374850.0625,
 'Total': 531420.9375}

In [88]:
print('A % contribution: ', SS['A']/SS['Total'])
print('C % contribution: ', SS['C']/SS['Total'])
print('AC % contribution: ', SS['AC']/SS['Total'])

A % contribution:  0.07773604610751716
C % contribution:  0.7053731534617979
AC % contribution:  0.17764178232062977


## Computational ANOVA

In [91]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

formula = 'Rate ~ A*B*C'
model = ols(formula, data=data).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table)

               sum_sq   df           F    PR(>F)
A          41310.5625  1.0   18.339364  0.002679
B            217.5625  1.0    0.096584  0.763911
A:B         2475.0625  1.0    1.098776  0.325168
C         374850.0625  1.0  166.410505  0.000001
A:C        94402.5625  1.0   41.908965  0.000193
B:C           18.0625  1.0    0.008019  0.930849
A:B:C        126.5625  1.0    0.056186  0.818586
Residual   18020.5000  8.0         NaN       NaN


Checks out. Now for pure model to calculate R^2

In [102]:
formula = 'Rate ~ A + C + A:C'
pure_model = ols(formula, data=data).fit()
aov_table = anova_lm(pure_model, typ=2)
ss = aov_table.sum_sq
s[0]

A            41310.5625
C           374850.0625
A:C          94402.5625
Residual     20857.7500
Name: sum_sq, dtype: float64

In [104]:
print('R^2: ', (ss[0]+ss[1]+ss[2])/ss.sum())

R^2:  0.96075098189


## Single Replicate

Due to *Sparsity of Effects* principle, you can combine higher order effects to calculate the error.

Or use *Lenth's Method*, which uses multiples of the median effect sizes
